In [150]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


from pathlib import Path
from typing import Tuple

In [151]:
def _CYME(df: pd.DataFrame) -> float:
    """ Compute the CYME metric, that is 1/2(median(yearly error) + median(monthly error))"""

    yearly_agg = df.groupby("cluster_nl")[["target", "prediction"]].sum().reset_index()
    yearly_error = abs((yearly_agg["target"] - yearly_agg["prediction"])/yearly_agg["target"]).median()

    monthly_error = abs((df["target"] - df["prediction"])/df["target"]).median()

    return 1/2*(yearly_error + monthly_error)


def _metric(df: pd.DataFrame) -> float:
    """Compute metric of submission.

    :param df: Dataframe with target and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])

    # Split 0 actuals - rest
    zeros = df[df["zero_actuals"] == 1]
    recent = df[df["zero_actuals"] == 0]

    # weight for each group
    zeros_weight = len(zeros)/len(df)
    recent_weight = 1 - zeros_weight

    # Compute CYME for each group
    return round(recent_weight*_CYME(recent) + zeros_weight*min(1,_CYME(zeros)),8)


def compute_metric(submission: pd.DataFrame) -> Tuple[float, float]:
    """Compute metric.

    :param submission: Prediction. Requires columns: ['cluster_nl', 'date', 'target', 'prediction']
    :return: Performance metric.
    """

    submission["date"] = pd.to_datetime(submission["date"])
    submission = submission[['cluster_nl', 'date', 'target', 'prediction', 'zero_actuals']]

    return _metric(submission)

In [152]:
# Define file paths

features_cols = [
    "brand",
    "che_pc_usd",
    "che_perc_gdp",
    "corporation",
    "country",
    "launch_date",
    "drug_id",
    "ind_launch_date",
    "indication",
    "insurance_perc_che",
    "population",
    "prev_perc",
    "price_month",
    "price_unit",
    "public_perc_che",
    "therapeutic_area",
    "Country_Group",
    "Price_Group",
    "indication_number",
    "avg_price_per_year",
    "month_number"
]
target_col = "target"
id_col = ["date","cluster_nl"]

base_dir = os.path.join(os.path.dirname(os.getcwd()), "dataset")
therap = os.path.join(os.path.dirname(os.getcwd()), "dataset", "therapeutic_area")
# Load datasets
# data = pd.read_csv(f"{base_dir}/train_data.csv", usecols=features_cols + [target_col] + id_col)
data = {}
y = {}
therapeutic_areas = os.listdir(therap)
extracted_parts = [name.split('_')[1].split('.')[0] for name in therapeutic_areas]
print(therapeutic_areas)

for i in extracted_parts:
    data[i] = pd.read_csv(f"{therap}/subset_{i}.csv", usecols=features_cols + [target_col] + id_col)
    print(f"Data loaded for therapeutic area {i}")
    y[i] = data[i][target_col]

test_data = pd.read_csv(f"{base_dir}/submission_data_TRY2.csv", usecols=features_cols + id_col)



['subset_980E.csv', 'subset_96D7.csv', 'subset_8E53.csv', 'subset_6CEE.csv', 'subset_CD59.csv', 'subset_051D.csv', 'subset_032C.csv', 'subset_4BA5.csv', 'subset_22ED.csv', 'subset_644A.csv', 'subset_66C5.csv', 'subset_645F.csv']
Data loaded for therapeutic area 980E
Data loaded for therapeutic area 96D7
Data loaded for therapeutic area 8E53
Data loaded for therapeutic area 6CEE
Data loaded for therapeutic area CD59
Data loaded for therapeutic area 051D
Data loaded for therapeutic area 032C
Data loaded for therapeutic area 4BA5
Data loaded for therapeutic area 22ED
Data loaded for therapeutic area 644A
Data loaded for therapeutic area 66C5
Data loaded for therapeutic area 645F


In [153]:
numeric_features = {}
categorical_features = {}
for i in extracted_parts:
    # convert int64 to float64
    data[i] = data[i].astype({"Country_Group": "float64"})
    data[i] = data[i].astype({"Price_Group": "float64"})
    data[i] = data[i].astype({"indication_number": "float64"})
    numeric_features[i] = data[i].select_dtypes(include=['float64']).drop(columns=[target_col], errors='ignore').columns
    categorical_features[i] = data[i].select_dtypes(include=['object']).columns

test_data = test_data.astype({"Country_Group": "float64"})
test_data = test_data.astype({"Price_Group": "float64"})
test_data = test_data.astype({"indication_number": "float64"})

# Separate numeric and categorical features for imputation


In [154]:
X = {}
for i in extracted_parts:
    # Drop unnecessary columns
    X[i] = data[i].drop(columns=[target_col]+["cluster_nl"])

X_test = test_data.drop(columns=["cluster_nl"])

# Preprocessing pipeline
def preprocess_data(X, preprocessor=None, fit=True):
    numerical_features = X.select_dtypes(include=['float64']).columns
    categorical_features = X.select_dtypes(include=['category']).columns

    if preprocessor is None:
        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

    if fit:
        X_transformed = preprocessor.fit_transform(X)
    else:
        X_transformed = preprocessor.transform(X)

    X_transformed = np.array(X_transformed)

    return X_transformed, preprocessor

# Preprocess data
X_transformed = {}
for i in extracted_parts:
    X_transformed[i], preprocessor = preprocess_data(X[i], fit=True)


X_test_transformed = preprocess_data(X_test, preprocessor=preprocessor, fit=False)

In [155]:
X_train = {}
X_valid = {}
y_train = {}
y_valid = {}
for i in extracted_parts:
    print(f"X_transformed shape for {i}: {X_transformed[i].shape}")
    X_train[i], X_valid[i], y_train[i], y_valid[i] = train_test_split(X_transformed[i], y[i], test_size=0.05, random_state=42)


X_transformed shape for 980E: (20298, 12)
X_transformed shape for 96D7: (45858, 12)
X_transformed shape for 8E53: (1523, 12)
X_transformed shape for 6CEE: (11871, 12)
X_transformed shape for CD59: (4578, 12)
X_transformed shape for 051D: (846, 12)
X_transformed shape for 032C: (2011, 12)
X_transformed shape for 4BA5: (1628, 12)
X_transformed shape for 22ED: (112, 11)
X_transformed shape for 644A: (7579, 12)
X_transformed shape for 66C5: (22024, 12)
X_transformed shape for 645F: (589, 12)


In [ ]:
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

models = {}
for i in extracted_parts:
    models[f'XGBoost_{i}'] = XGBRegressor(n_estimators=100, random_state=42)
    models[f'LightGBM_{i}'] = lgb.LGBMRegressor(n_estimators=100, random_state=42)
    models[f'RandomForest10_{i}'] = RandomForestRegressor(n_estimators=10, random_state=42)
    models[f'RandomForest100_{i}'] = RandomForestRegressor(n_estimators=100, random_state=42)
    models[f'RandomForest200_{i}'] = RandomForestRegressor(n_estimators=200, random_state=42)

print(models)
results = {}

print(X_train.keys())
for name, model in models.items():
    i = name.split('_')[1]
    model.fit(X_train[i], y_train[i])
    scores = cross_val_score(model, X_train[i], y_train[i], scoring='neg_root_mean_squared_error', cv=5)
    results[name] = -scores.mean()


{'XGBoost_980E': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...), 'LightGBM_980E': LGBMRegressor(random_state=42), 'RandomForest10_980E': RandomForestRegressor(n_estimators=10, random_state=42), 'RandomForest100_980E': RandomForestRegressor(random_state=42), 'RandomForest200_980E': RandomForestRegressor

In [ ]:
best_model = {}
best_results = {}

for i in extracted_parts:
    best_model[i] = None
    # get name of models
    name_models = [name.split("_")[0] for name in results.keys() if name.endswith(i)]
    print(name_models)
    # get best model for each therapeutic area by name
    best_model[i] = min(name_models, key=lambda x: results[x + "_" + i])
    print(f"Best model for {i}: {best_model[i]}, RMSE: {results[best_model[i] + '_' + i]}")
    best_results[best_model[i] + '_' + i] = results[best_model[i] + '_' + i]
    # remove other models for this therapeutic area
    for name in name_models:
        if name != best_model[i]:
            del models[name + "_" + i]
    
best_results
# make it a dataframe
best_results_df = pd.DataFrame(best_results.items(), columns=['model', 'RMSE'])
best_results_df

In [ ]:
print(results)
# Display RMSE results
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE']).sort_values(by='RMSE')
results_df['RMSE'] = results_df['RMSE'].map("{:.5f}".format)
results_df

{'LightGBM_980E': 0.046805352069617875, 'XGBoost_980E': 0.04059499694921115, 'RandomForest10_980E': 0.04098139248628171, 'RandomForest100_980E': 0.04032042918686416, 'RandomForest200_980E': 0.040319883676993074, 'LightGBM_96D7': 0.12540737524099815, 'XGBoost_96D7': 0.09161553146302344, 'RandomForest10_96D7': 0.08571232810695441, 'RandomForest100_96D7': 0.08346607559505617, 'RandomForest200_96D7': 0.08351053368344968, 'LightGBM_8E53': 0.03567071254036328, 'XGBoost_8E53': 0.03693772066170276, 'RandomForest10_8E53': 0.03814311628936089, 'RandomForest100_8E53': 0.036867333129258266, 'RandomForest200_8E53': 0.03694131809325557, 'LightGBM_6CEE': 0.05626180170717787, 'XGBoost_6CEE': 0.05063509130761432, 'RandomForest10_6CEE': 0.052078871820508445, 'RandomForest100_6CEE': 0.05099505506499512, 'RandomForest200_6CEE': 0.050895737619837865, 'LightGBM_CD59': 0.06456791679116328, 'XGBoost_CD59': 0.06729711343413844, 'RandomForest10_CD59': 0.06584272034087764, 'RandomForest100_CD59': 0.0636180950479

,Model,RMSE
44,RandomForest200_22ED,0.00338
43,RandomForest100_22ED,0.00338
40,LightGBM_22ED,0.00341
41,XGBoost_22ED,0.00342
42,RandomForest10_22ED,0.00348
55,LightGBM_645F,0.02935
59,RandomForest200_645F,0.02976
58,RandomForest100_645F,0.02989
57,RandomForest10_645F,0.03172
56,XGBoost_645F,0.03186


In [ ]:
X_test_transformed = preprocess_data(X_test, preprocessor=preprocessor, fit=False)
print(X_test_transformed[0])

# Ensure indices align between test_data and X_test_transformed
X_test_transformed = pd.DataFrame(X_test_transformed[0], index=test_data.index)  # Specify column names if necessary

def extract_part(text):
    return text.split('_')[2].split('.')[0]

X_test_transformed['therapeutic_area'] = test_data['therapeutic_area'].apply(extract_part)
print(X_test_transformed)

# Initialize a DataFrame to store the results
results_final = pd.DataFrame()

# Iterate over each therapeutic area
for area in models.keys():
    keyy = area.split('_')[1]
    print(keyy)
    # Filter test data for the current therapeutic area
    area_indices = X_test_transformed[X_test_transformed['therapeutic_area'] == keyy].index
    X_area = X_test_transformed.loc[area_indices].drop('therapeutic_area', axis=1)

    # Get the corresponding model
    model = models[area]
    print(models)

    # Make predictions
    if not X_area.empty:
      predictions = model.predict(X_area)

      # Prepare the result DataFrame
      test_area = test_data.loc[area_indices]
      area_results = pd.DataFrame({
          'date': pd.to_datetime(test_area['date']).dt.strftime("%m/%d/%Y"),
          'cluster_nl': test_area['cluster_nl'],
          'prediction': predictions
      })

      # Append to the results
      results_final = pd.concat([results_final, area_results], ignore_index=True)

results_final.to_csv('result.csv', index=False)
print("Result file saved as 'result.csv'")

[[ 0.89587136 -2.0850563  -0.45612093 ...  0.88591675 -1.
  -1.43190039]
 [ 0.89587136 -2.0850563  -0.45612093 ...  0.88591675 -1.
  -1.279425  ]
 [-1.51092879 -4.38200642 -0.00976256 ...  0.88591675  0.
  -1.45829596]
 ...
 [-1.12819784 -1.92214442  1.14928472 ...  0.88591675 -1.
  -0.17857789]
 [-1.12819784 -1.92214442  1.14928472 ...  0.88591675 -1.
  -0.26626926]
 [-1.50954928 -4.25542441 -0.30495977 ...  0.88591675 -1.
   0.12994308]]
             0         1         2         3         4         5         6  \
0     0.895871 -2.085056 -0.456121 -0.783378 -0.434106  0.791603 -1.292750   
1     0.895871 -2.085056 -0.456121 -0.783378  0.221657 -1.012921 -1.095225   
2    -1.510929 -4.382006 -0.009763  3.943438 -0.392071 -1.012921 -1.305431   
3    -1.121116 -1.782185  1.090222 -0.703284 -0.750251 -1.012921 -0.825598   
4    -1.509549 -4.255424 -0.304960  1.068477 -0.774814 -1.012921  0.621289   
...        ...       ...       ...       ...       ...       ...       ...   
1764  2.32